Exercise 1. American Roulette probabilities

An American roulette wheel has 18 red, 18 black, and 2 green pockets. Each red and black pocket is associated with a number from 1 to 36. The two remaining green slots feature "0" and "00". Players place bets on which pocket they think a ball will land in after the wheel is spun. Players can bet on a specific number (0, 00, 1-36) or color (red, black, or green).

What are the chances that the ball lands in a green pocket?

In [8]:
p_green = 2 / (18 + 18 + 2)
print(p_green)

0.05263157894736842


Exercise 2. American Roulette payout

In American roulette, the payout for winning on green is $17. This means that if you bet $1 and it lands on green, you get $17 as a prize.

Create a model to predict your winnings from betting on green one time.

In [18]:
import pyro
import torch

def greenWin():
    return torch.where(pyro.sample("greenWin", pyro.distributions.Binomial(probs=p_green)) == 1.0, torch.tensor(17.0), torch.tensor(-1.0))

Exercise 3. American Roulette expected value

In American roulette, the payout for winning on green is $17. This means that if you bet $1 and it lands on green, you get $17 as a prize.In the previous exercise, you created a model to predict your winnings from betting on green.

Now, compute the expected value of X
, the random variable you generated previously.

In [20]:
import torch
import pyro

def getSampleExpectedWinning():
    with pyro.plate("greenWin", 50000):
        return torch.mean(greenWin())
    
sample_exp_win = getSampleExpectedWinning().item()
comp_exp_win = 17 * p_green + -1 * (1 - p_green)

print(sample_exp_win)
print(comp_exp_win)

-0.06723999977111816
-0.052631578947368585


Exercise 4. American Roulette standard error

The standard error of a random variable X
tells us the difference between a random variable and its expected value. You calculated a random variable X

in exercise 2 and the expected value of that random variable in exercise 3.

Now, compute the standard error of that random variable, which represents a single outcome after one spin of the roulette wheel.

In [22]:
import math
import torch

def computeSEfromSamples(expected):
    with pyro.plate("greenWin", 50000):
        samples = greenWin()
        s1 = samples - expected
        s2 = s1 ** 2
        return math.sqrt(torch.mean(s2))

standard_error_sampled = computeSEfromSamples(sample_exp_win)
standard_error_green_win = 18 * math.sqrt(p_green * (1 - p_green))
print(standard_error_sampled)
print(standard_error_green_win)

3.961460899066654
4.019343808849849


Exercise 5. American Roulette sum of winnings

You modeled the outcome of a single spin of the roulette wheel, X

, in exercise 2.

Now create a random variable S
that sums your winnings after betting on green 1,000 times.

In [23]:
n = 1000
expected_win_sum = n * comp_exp_win

Exercise 6. American Roulette winnings expected value

In the previous exercise, you generated a vector of random outcomes, S

, after betting on green 1,000 times.

What is the expected value of S
?

In [24]:
print(expected_win_sum)

-52.63157894736858


Exercise 7. American Roulette winnings expected value

You generated the expected value of S

, the outcomes of 1,000 bets that the ball lands in the green pocket, in the previous exercise.

What is the standard error of S
?

In [25]:
standard_error_win_sum = math.sqrt(n) * standard_error_green_win
print(standard_error_win_sum)

127.10281135261964


Exercise 1. American Roulette probability of winning money

The exercises in the previous chapter explored winnings in American roulette. In this chapter of exercises, we will continue with the roulette example and add in the Central Limit Theorem.

In the previous chapter of exercises, you created a random variable S

that is the sum of your winnings after betting on green a number of times in American Roulette.

What is the probability that you end up winning money if you bet on green 100 times?

In [29]:
import pyro
import math

green_bet_win_dist = pyro.distributions.Normal(torch.tensor(100 * comp_exp_win), torch.tensor(math.sqrt(100) * standard_error_green_win))

print(100 * comp_exp_win)
print(math.sqrt(100) * standard_error_green_win)

p_winning_money = 1 - green_bet_win_dist.cdf(0)
print(p_winning_money)

-5.2631578947368585
40.193438088498496
tensor(0.4479)


Exercise 2. American Roulette Monte Carlo simulation

Create a Monte Carlo simulation that generates 10,000 outcomes of S

, the sum of 100 bets.

Compute the average and standard deviation of the resulting list and compare them to the expected value (-5.263158) and standard error (40.19344) for S
that you calculated previously.

In [32]:
import pyro
import torch

def betting100Times():
    with pyro.plate("100times", 100):
        return torch.sum(greenWin())

samples = []
for _ in range(10000):
    samples.append(betting100Times())

sampled_win_sum_avg = torch.mean(torch.tensor(samples))
sampled_win_sum_sdev = torch.std(torch.tensor(samples))

print(sampled_win_sum_avg)
print(sampled_win_sum_sdev)

tensor(-5.5558)
tensor(39.8359)


Exercise 3. American Roulette Monte Carlo vs CLT

In this chapter, you calculated the probability of winning money in American roulette using the CLT.

Now, calculate the probability of winning money from the Monte Carlo simulation. The Monte Carlo simulation from the previous exercise has already been pre-run for you, resulting in the variable S that contains a list of 10,000 simulated outcomes.

In [34]:
import torch

samples_wins_greater_zero = torch.where(torch.tensor(samples) > 0.0, torch.ones(1), torch.zeros(1)).nonzero().size()[0]

p_win_money_sampled = samples_wins_greater_zero / 10000
print(p_win_money_sampled)

0.4269
